In [ ]:
import pandas as pd
import openpyxl as op
import cx_Oracle as cx
import mysql.connector as mysql
from datetime import date
from dateutil.relativedelta import relativedelta
from calendar import monthlen

In [ ]:
"""
- Aqui são utilizadas as bibliotecas monthlen e relativedelta para definir datas de início e final.
- A data nicial é definida como o primeiro dia do mês anterior e a data final o último dia do mês anterior.
- A frequência desta atualização é mensal, ocorrendo no segundo dia de cada mês.
"""
d = date.today()
StartDate = d.replace(day=1) - relativedelta(months=1)
EndDate = d.replace(day=monthlen(d.year,d.month)) - relativedelta(months=1)

- A célula abaixo define a consulta de extração que será executada no banco de dados transacional que está atualmente em produção
Nesta consulta são utilizados os parâmetros de data incial e final defindos na célula anterior.

Como há na organização mais que um software que faça movimentações de produtos, foi necessário, no início unir
consultas que extraíam dados com tipo de lançamentos diferentes.

De modo geral, os tipos variavam com junto com a natureza do movimento, que pode ser de entrada ou saída.
Isso fazia com que, caso fosse requisitada uma natureza ou outra, o tipo fosse retornado nulo
Todavia, com a recente alteração da extração do tipo de lançamento, isso pode e está sendo resolvido com a função coalesce.

Uma próxima alteração neste código reduzirá pela metade sua extensão, o que o tornará mais legível.
Será registrada também a variação no desempenho da consulta.

In [ ]:
consulta = f"""

--ENTRADAS NA TROCA

SELECT

A.NROEMPRESA,
A.DTAENTRADASAIDA,
A.SEQPRODUTO,
A.CODGERALOPER,
B.DESCRICAO,
ROUND(SUM(A.QTDENTRADAOUTRAS), 2) AS QTD,
ROUND(SUM(A.VLRENTRADAOUTRAS), 2) AS VLR,
COALESCE(
    (SELECT MAX(C.TIPLANCTO) FROM XXXXXX.CGESMOV_PRODUTOS_COLETADOS C WHERE A.NROEMPRESA = C.NROEMPRESA AND A.SEQPRODUTO = C.SEQPRODUTO AND TO_DATE(A.DTAENTRADASAIDA, 'dd/mm/yyyy') = TO_DATE(C.DATA_COLETA, 'dd/mm/yyyy')),
    A.MOTIVOMOVTO) AS TIPLANCTO,
A.USULANCTO,
'E' AS NATMOVTO

    FROM CONSINCO.MAXV_ABCMOVTOBASE_PROD A
    LEFT JOIN CONSINCO.GE_CGO B ON A.CODGERALOPER = B.CGO

WHERE A.DTAENTRADASAIDA BETWEEN TO_DATE('{StartDate}', 'yyyy-mm-dd') AND TO_DATE('{EndDate}', 'yyyy-mm-dd')
AND A.DESCRICAOLOCAL = 'TROCA'
AND A.TIPLANCTO = 'E'

    GROUP BY A.DTAENTRADASAIDA, A.NROEMPRESA, A.SEQPRODUTO, A.DTAENTRADASAIDA, A.CODGERALOPER, A.USULANCTO, B.DESCRICAO, A.MOTIVOMOVTO

-- COMBINAÇÃO
    UNION

-- SAIDAS DA TROCA
SELECT

A.NROEMPRESA,
A.DTAENTRADASAIDA,
A.SEQPRODUTO,
A.CODGERALOPER,
B.DESCRICAO,
ROUND(SUM(A.QTDSAIDAOUTRAS), 2) AS QTD,
ROUND(SUM(A.VLRSAIDAOUTRAS), 2) AS VLR,
COALESCE(
    (SELECT MAX(C.TIPLANCTO) FROM XXXXXX.CGESMOV_PRODUTOS_COLETADOS C WHERE A.NROEMPRESA = C.NROEMPRESA AND A.SEQPRODUTO = C.SEQPRODUTO AND TO_DATE(A.DTAENTRADASAIDA, 'dd/mm/yyyy') = TO_DATE(C.DATA_COLETA, 'dd/mm/yyyy')),
    A.MOTIVOMOVTO) AS TIPLANCTO,
A.USULANCTO,
'S' AS NATMOVTO

    FROM CONSINCO.MAXV_ABCMOVTOBASE_PROD A
    LEFT JOIN CONSINCO.GE_CGO B ON A.CODGERALOPER = B.CGO

WHERE A.DTAENTRADASAIDA BETWEEN TO_DATE('{StartDate}', 'yyyy-mm-dd') AND TO_DATE('{EndDate}', 'yyyy-mm-dd')
AND A.DESCRICAOLOCAL = 'TROCA'
AND A.TIPLANCTO = 'S'

    GROUP BY A.DTAENTRADASAIDA, A.NROEMPRESA, A.SEQPRODUTO, A.DTAENTRADASAIDA, A.CODGERALOPER, A.USULANCTO, B.DESCRICAO, A.MOTIVOMOVTO
"""

###### - Como neste processo serão extraídos e transformados dados de uma base Oracle e inseridos em um banco de dados em MySQL que está disponível aos times de negócio. São utilizadas as bibliotecas cx_Oracle e mysql.connector

In [ ]:
#Cria conexão no database Oracle
ora_conn = cx.connect(user = 'usuario', password = 'senha123', dsn = '000.000.0.000/dbname')
ora_cursor = ora_conn.cursor()

#Cria conexão no database MySQL
my_conn = mysql.connect(user = 'usuario', password = 'senha123', host = '000.000.0.000', database = 'db_name')
my_cursor = my_conn.cursor()

# ! Por motivos óbvios, os parâmetros de conexão e credenciais foram substituídos por fictícios

In [ ]:
#Executa consulta no database Oracle
df = ora_cursor.execute(consulta).fetchall()

###### - Como toda a parte de transformação deste ETL é feito em SQL, a execução da consulta e a inserção dos dados no banco MySQL é feita de forma direta

In [ ]:
#Executa comando de inserção de dados no database MySQL
for linha in df:
    nroempresa = linha[0]
    dtaentradasaida = linha[1]
    seqproduto = linha[2]
    coderaloper = linha[3]
    descmovto = linha[4]
    qtd = linha[5]
    vlr = linha[6]
    idtiplancto = linha[7]
    usulancto = linha[8]
    natmovto = linha[9]
    my_cursor.execute(f"INSERT INTO prev_perdas.movimentacoes (nroempresa, dtaentradasaida, seqproduto, codgeraloper, descricao, qtd, vlr, idtiplancto, usulancto, naturezamovto) VALUES ('{nroempresa}', '{dtaentradasaida}', '{seqproduto}', '{coderaloper}', '{descmovto}', '{qtd}', '{vlr}', '{idtiplancto}', '{usulancto}', '{natmovto}')")
my_conn.commit()